# Setup

In [1]:
import numpy as np
import math

Need to use `.item()` as per https://stackoverflow.com/questions/24565916/why-is-numpy-shape-empty (and also Frank's message)

Also note that `mat1` contains only protons, and `mat2` contains only iron nuclei.

In [2]:
mat1 = np.load('../data/sim_12360_00.npy').item()
mat2 = np.load('../data/sim_12362_00.npy').item()

# Exploration

In [3]:
mat1.keys()

dict_keys(['Charges', 'Energy', 'File_info', 'Gain', 'Position', 'Fit_status', 'Composition'])

Each index in each array corresponds to one cosmic ray.
* Charges: dictionaries, where each dictionary has about (but not exactly!) 6-8 values. Each key is a sensor and each value is the value that sensor gathered. Presumably the sensors that did not recieve significant signal are not included.
* Energy: one value per event. Unclear what that value is.
* File_info: some information about the file that the event was drawn from 
* Gain: a list with one item-- a dictionary from sensor to whether it is a high-gain or low-gain sensor. 
* Position: the position of each sensor. Consists of an x-position array and a y-position array, but it is unclear what order they are in. 
* Fit_status: mostly `'OK'` with some `'InsufficientHits'`. Unclear where that comes into play.
* Composition: a string representing the composition of each ray.

In [4]:
len(mat1['Gain'])

1

In [5]:
len(mat1['File_info'])

16531

In [6]:
sorted(mat1['Position'][1].values())

[-497.8949890136719,
 -497.8949890136719,
 -496.07000732421875,
 -496.07000732421875,
 -477.76499938964844,
 -477.76499938964844,
 -476.5749969482422,
 -476.5749969482422,
 -464.49000549316406,
 -464.49000549316406,
 -458.9750061035156,
 -458.9750061035156,
 -438.7050018310547,
 -438.7050018310547,
 -436.864990234375,
 -436.864990234375,
 -432.4100036621094,
 -432.4100036621094,
 -424.68499755859375,
 -424.68499755859375,
 -400.4750061035156,
 -400.4750061035156,
 -399.0050048828125,
 -399.0050048828125,
 -398.3800048828125,
 -398.3800048828125,
 -398.1300048828125,
 -398.1300048828125,
 -389.260009765625,
 -389.260009765625,
 -382.94500732421875,
 -382.94500732421875,
 -362.44000244140625,
 -362.44000244140625,
 -359.7300109863281,
 -359.7300109863281,
 -353.385009765625,
 -353.385009765625,
 -343.8999938964844,
 -343.8999938964844,
 -343.47999572753906,
 -343.47999572753906,
 -342.4700012207031,
 -342.4700012207031,
 -339.7449951171875,
 -339.7449951171875,
 -335.68499755859375,
 -33

## Questions

* Do the x/y positions correlate to some real-world data? Is it possible to, for example, normalize to all positive?
* What does `mat1['File_info']` contain? My instinct is some information about the original simulation files that this was drawn from but perhaps not.
* Is it the case that sensors that did not see any signal are not included in `'Charges'`?
* What is stored under `Energy`?
* Is the `position` array `[x_dict, y_dict]` or `[y_dict, x_dict]`?
* Do we have access to the actual/calculated initial conditions information (angle/center) or is that something we need to calculate? If the latter, is there a script somewhere?

# Build a gain-differentiated dataset

Find all the sensors that are high/low gain

In [7]:
hgain = list(filter(lambda x: mat1['Gain'][0][x] == 'High', mat1['Position'][0].keys()))
lgain = list(filter(lambda x: mat1['Gain'][0][x] == 'Low', mat1['Position'][0].keys()))

Construct a dictionary that goes from name -> column index in the matrix we're about to build

In [8]:
#build a name->index dict
def get_index_dict(sensors):
    name_index_dict = {}
    for i in range(len(hgain)):
        name_index_dict[hgain[i]] = i
    return name_index_dict

In [9]:
hgain_indices = get_index_dict(hgain)
lgain_indices = get_index_dict(lgain)

Build the matrix by setting up an empty one and populating it with the values according to the indices we created above. This is pretty slow atm but I'm sure there are ways to make it more efficient. Perhaps we don't need to make it dense in order to pop it into numpy? 

In [13]:
hgain_events = np.zeros((len(mat1['Composition']) + len(mat2['Composition']), len(hgain_indices) + 1))
for i in range(len(mat1['Charges'])):
    event = mat1['Charges'][i]
    for sensor in mat1['Charges'][i].keys():
        try:
            hgain_events[i, hgain_indices[sensor]] = 0 if math.isnan(event[sensor]) else event[sensor]
            # print('updated x: {} y: {} to be {}'.format(i,hgain_indices[sensor],event[sensor]))
        except KeyError:
            continue
    hgain_events[i, -1] = 1

for i in range(len(mat2['Charges'])):
    event = mat2['Charges'][i]
    for sensor in mat2['Charges'][i].keys():
        try:
            hgain_events[len(mat1['Charges']) + i, hgain_indices[sensor]] = 0 if math.isnan(event[sensor]) else event[sensor]
        except KeyError:
            continue
    hgain_events[len(mat1['Charges']) + i, -1] = 0

In [12]:
mat1['Charges'][102]

{'2163': 1.1980705261230469,
 '3061': 1.4387215375900269,
 '3063': 2.461458921432495,
 '3861': 0.6513364315032959,
 '3863': 0.3660579025745392,
 '3962': nan,
 '3963': 9.770613670349121,
 '4061': 8.847500801086426,
 '4063': 5.4975905418396,
 '4861': 0.5231978893280029,
 '4863': 0.38361236453056335,
 '4961': 4.835577964782715,
 '4963': 2.4362757205963135,
 '5861': 0.3139652609825134,
 '5863': 0.3876802325248718}

In [ ]:
for key in mat1['Charges'][0].keys():
    print(key)
    print(str(hgain_events[0,hgain_indices[key]]) + " " + str(mat1['Charges'][0][key]))

In [ ]:
hgain_events[0,140]

In [ ]:
hgain_events[16529, :]

# Split high-gain into training/test set

In [16]:
np.random.shuffle(hgain_events)
train_size = int(hgain_events.shape[0] * .9)

trainset = hgain_events[:train_size]
testset = hgain_events[train_size:]

In [17]:
trainset[trainset[:,-1]==0].shape

(13601, 163)

In [18]:
np.save('small_train.npy', trainset)
np.save('small_test.npy', testset)